In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from torch.autograd import Variable
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import random
import math
def seed_all(seed):
    if not seed:
        seed = 10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_all(1)

In [ ]:
dataset = pd.read_csv('./iris.csv')

# transform species to numerics
dataset.loc[dataset.variety == 'Setosa', 'variety'] = 0
dataset.loc[dataset.variety == 'Versicolor', 'variety'] = 1
dataset.loc[dataset.variety == 'Virginica', 'variety'] = 2

train_X, test_X, train_y, test_y = train_test_split(dataset[dataset.columns[0:4]].values,
                                                    dataset.variety.values, test_size=0.2)

tensor_train_x = Variable(torch.Tensor(train_X).float())
tensor_train_y = Variable(torch.Tensor(list(train_y)).long())

tensor_test_x = Variable(torch.Tensor(test_X).float())
tensor_test_y = Variable(torch.Tensor(list(test_y)).long())

In [ ]:
path='./iris_model.pt'

In [ ]:
def space_space_generator(action, length):
    res= length *'0'+(bin(int(action))[2:])
    if len(res) > length:
        res=res[-length:]      
    return res

In [ ]:
def get_accuracy(x,y,model):
    test_pred=model(x)
    _, predict_y = torch.max(test_pred, 1)
    return accuracy_score(y, predict_y)

In [ ]:
def maxAction(Q, state, actions):
#     print(actions, state,Q )
#     x=input()
    values = np.array([Q[state,a] for a in actions])
    action = np.argmax(values)
#     print(actions, state)
    
    return actions[action]

In [ ]:
def adjust_weight(state_binary, weight, original):
    w=[len(i) for i in original] 
#     print('\n\n\n In binary', state_binary,'\n\n curr_w : ', weight, '\n\n orignial w', original)
    binaries=[]
    
    number_weights=len(w)
    
    counter=0
    for i in w:
        binaries.append(state_binary[counter:counter+i])
        counter=i
     
    for i,j,k in zip(weight,binaries, original):
        for y in range(len(i)):
            i[y]=0 if j[y]=='0' else k[y]
     

#     print('\n\n\n Out binary', state_binary,'\n\n curr_w : ', weight, '\n\n orignial w', original)

In [ ]:
class ModelWorld(object):
    
    def __init__(self,  err, model_path, x_test, test_y):
        
        self.x_test=x_test
        self.model = torch.load(model_path)
        self.compressed_model = torch.load(model_path)
        
        self.test_y=test_y
        
        self.number_of_layers=0
        self.number_of_parameters= 0
        
        self.o_weights=[]
        self.all_shapes=[]        
        self.c_weights=[]
        
        for name, param in self.compressed_model.named_parameters():  
            if 'weight' in name:
                self.c_weights.append(param.detach().flatten())
                
            
        for name, param in self.model.named_parameters():            
            
            if 'weight' in name:       
                self.number_of_layers+=1
                self.number_of_parameters+=len(param.detach().flatten())
                
                self.o_weights.append(param.detach().flatten())
                self.all_shapes.append(param.shape)
   
        
        self.achived_acc = 0
        self.acc = get_accuracy(self.x_test,self.test_y,self.model)
        
        self.err=err
        
        self.total_space=math.pow(2,self.number_of_parameters)
        
        self.action_space=[]
        
        for layer in range(self.number_of_layers):
            dim=len(self.c_weights[layer])
            self.action_space+=[(layer,i,'1') for i in range(dim)]+ [(layer,i,'0') for i in range(dim)]

  
#         self.weights = self.compressed_model[0].state_dict()['weight']
      
        
        # since there are N  weight's , the space is a string of N. Then all combinations of N is the posible states.
#         for layer in range(self.number_of_layers):
        self.stateSpace =[space_space_generator(i, self.number_of_parameters) for i in range(int(self.total_space))] 
        
        self.current_state='0'*self.number_of_parameters
        
        adjust_weight(self.current_state,self.c_weights, self.o_weights)


    
    def action_space_sample(self): # return random stuff here
        return self.action_space[np.random.randint(len(self.action_space))]
    
    def take_action(self,action):
#         print("In state : ", self.current_state, 'in action: ',action, " : in wieght :", self.c_weights )
        
        temp=list(self.current_state)        
        
        temp[action[1]]=action[2]
        temp=''.join(temp)

        self.current_state=temp

        adjust_weight(self.current_state,self.c_weights, self.o_weights)
    
#         print(" out state : ", self.current_state, "Out weight", self.c_weights)
         
        
        self.achived_acc = get_accuracy(self.x_test,self.test_y, self.compressed_model)
    
    def legal_move(self,action):
        return not (self.current_state[action[1]]==action[2])
        
    def step(self, action):        
        done = bool ((np.abs(self.acc - self.achived_acc) <= self.err) or \
                     self.current_state.count('1') == self.number_of_parameters)
    
        added= 1 if action[2] == '1' else 0
        if self.legal_move(action):

            self.take_action(action)

            reward = self.achived_acc -added
            
            
        else:

            reward = -2

            
        return self.current_state, reward, done, None
 
    def reset(self):
        self.current_state='0'*self.number_of_parameters
        return self.current_state

In [ ]:
compress = ModelWorld(.1,path,tensor_test_x,tensor_test_y)

In [ ]:
compress.current_state

In [ ]:
compress.number_of_parameters

In [ ]:
if __name__ == '__main__':
    # model hyperparameters
    ALPHA = 0.1
    GAMMA = 1.0
    EPS = 1.0

    Q = {}
    for state in compress.stateSpace:
        for action in compress.action_space :
            Q[state, action] = 0


    numTrials = 5
    totalRewards = np.zeros(numTrials)
    total_played_per_game=[]
    for i in range(numTrials):        
#         if i % 10 == 0:
        print('start compress ', i)
        done = False
        epRewards = 0
        observation = compress.reset()
        actions_taken_per_game=0
  
        while not done: 
#             print(" State now is : ", compress.current_state/)
            w_percentage=(list(compress.current_state).count('1')/8) *100
            if actions_taken_per_game%50000==0:                 
                print("w : ",w_percentage,'% Acc: ',compress.achived_acc , ' state:', compress.current_state)
            actions_taken_per_game+=1
            rand = np.random.random()
            action = maxAction(Q,observation, compress.action_space) if rand < (1-EPS)  else compress.action_space_sample()

            observation_, reward, done, info = compress.step(action)
            epRewards += reward

            action_ = maxAction(Q, observation_, compress.action_space)
            
            Q[observation,action] = Q[observation,action] + ALPHA*(reward + \
                        GAMMA*Q[observation_,action_] - Q[observation,action])
            observation = observation_
 

        if EPS - 2 / numTrials > 0:
            EPS -= 2 / numTrials
        else:
            EPS = 0

        totalRewards[i] = epRewards
        total_played_per_game.append(actions_taken_per_game)
        actions_taken_per_game=0

    plt.plot(totalRewards)
    plt.show()

In [ ]:
total_played_per_game

In [ ]:
small_model = compress.compressed_model

In [ ]:
small_model[0].state_dict()['weight']

In [ ]:
test_pred=small_model(tensor_test_x)
_, predict_y = torch.max(test_pred, 1)
print(accuracy_score(tensor_test_y, predict_y))

In [ ]:
res=small_model(tensor_test_x)

In [ ]:
res.shape

In [ ]:
predict_y.shape

In [ ]:
total_played_per_game